# *Install Required libraries*

In [1]:
pip install numpy pandas scikit-learn

# *Prepare Data*

In [5]:
import numpy as np
import pandas as pd
data={
    'user_id':[1,1,1,2,2,2,3,3,3,4,4],
    'item_id':[101,102,103,101,103,104,102,104,105,103,105],
    'rating':[5,3,4,4,5,2,1,5,3,2,4]
}
df_ratings=pd.DataFrame(data)
ratings_matrix=df_ratings.pivot(index='user_id',columns='item_id',values='rating').fillna(0)
print(ratings_matrix)

item_id  101  102  103  104  105
user_id                         
1        5.0  3.0  4.0  0.0  0.0
2        4.0  0.0  5.0  2.0  0.0
3        0.0  1.0  0.0  5.0  3.0
4        0.0  0.0  2.0  0.0  4.0


# *Build Recommender System*

In [7]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=2)
user_factors=svd.fit_transform(ratings_matrix)
item_factors=svd.components_
predicted_ratings=np.dot(user_factors,item_factors)
predicted_ratings_df=pd.DataFrame(predicted_ratings,index=ratings_matrix.index,columns=ratings_matrix.columns)
print(predicted_ratings_df)

item_id       101       102       103       104       105
user_id                                                  
1        4.745798  1.570881  4.694097  0.306985 -0.314598
2        4.192479  1.514829  4.390191  1.307917  0.693323
3       -0.323673  0.397791  0.646751  4.097661  3.879912
4        0.542830  0.435686  1.027148  2.123306  1.920312


# **Make Recommendations**

In [8]:
def recommend_item(user_id,predicted_ratings_df,ratings_matrix,num_recommendations=2):
  user_ratings=ratings_matrix.loc[user_id]
  already_rated=user_ratings[user_ratings>0].index.tolist()
  sorted_ratings=predicted_ratings_df.loc[user_id].sort_values(ascending=False)
  recommendations=[item for item in sorted_ratings.index if item not in already_rated][:num_recommendations]
  return recommendations
recommendations=recommend_item(1,predicted_ratings_df,ratings_matrix)
print(f"Recommended items for user1:{recommendations}")


Recommended items for user1:[104, 105]


# *Explainability*

In [10]:
def explain_recommendation(user_id,item_id,user_factors,item_factors):
  user_latent_factors=user_factors[user_id -1]
  item_latent_factors=item_factors[:,item_id-ratings_matrix.columns[0]]
  score=np.dot(user_latent_factors,item_latent_factors)
  explanation = f"Item {item_id} is recommended to User {user_id} because of the following factors:\n" \
                  f"- User's preference profile: {user_latent_factors}\n" \
                  f"- Item's attribute profile: {item_latent_factors}\n" \
                  f"- Combined recommendation score: {score}"

  return explanation

# Explain why item 102 is recommended to user 1
explanation = explain_recommendation(1, 102, user_factors, item_factors)
print(explanation)

Item 102 is recommended to User 1 because of the following factors:
- User's preference profile: [ 6.60308005 -1.90193368]
- Item's attribute profile: [ 0.23551464 -0.00828565]
- Combined recommendation score: 1.5708807650605896
